<a href="https://colab.research.google.com/github/ChaitaliV/generative-explanation/blob/main/datacollection/unsupervised_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Create Unsupervised Dataset for Generative Transformer
* read pdfs and save the data in .txt file.
* manually add some more data to it, from blogs and other online resources which are not downloadable.
* manually read the data once, and remove text which is not relevant
* clean the text
* create sentences
* create unsupervised learning dataset for generative transformer model by selectively masking words in all sentences

In [3]:
!pip install pyPDF2 transformers sentencepiece
!git clone https://github.com/ChaitaliV/generative-explanation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
Cloning into 'generative-explanation'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 124 (delta 15), reused 3 (delta 3), pack-reused 96
Receiving objects: 100% (124/124), 6.84 MiB | 9.29 MiB/s, done.
Resolving deltas: 100% (50/50), done.


### Create Raw Dataset
* read text from the pdf which are textbooks, manuals and other web resources on depression and it's dignosis. this will be our raw data
* Go through the raw data manually, and add % to seperate topics, this should work better than seperating the text by sentences, as whole topics as input will retain long term dependencies.
* Also remove garbage text and topics that are not relevant.

In [ ]:
from PyPDF2 import PdfReader
import os

In [ ]:
folder_path = 'generative-explanation/datasets/unsupervised dataset'

In [ ]:
def read_pdf(file_path):
    """fn to read pdf files"""
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# get the list of all pdf from the folder
file_list = [file for file in os.listdir(folder_path) if file.endswith('.pdf')]
corpus = ''

# Iterate through PDF files and read text
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    text = read_pdf(file_path)
    corpus += text

#save the corpus as .txt file for future use
def save_to_txt(text, file_path):
    with open(file_path, 'w') as file:
        file.write(text)

save_to_txt(corpus, 'generative-explanation/raw_unsupervised_text.txt')

### Process the raw text to create Dataset with masked tokens
* load the raw data, remove numbers and special characters. and seperate the data from % token.
* mask words and create unsupervised dataset to predict the masked words.
* for masking process, mask individual masks

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import random
import numpy as np
import re
import pandas as pd

In [5]:
raw_data_path = 'generative-explanation/datasets/unsupervised dataset/raw_unsupervised_text.txt'

In [6]:
corpus = ''
with open(raw_data_path, 'r') as file:
  corpus = file.read()

In [20]:
class MaskData():

  def __init__(self,text, tokenizer):
    """
    masking methods is selected by either 0 or 1.
    0: masking of tokens
    1: masking of words
    """
    self.corpus = text
    self.tokenizer = tokenizer

  def mask_tokens(self, sentence, mask_percentage = 0.4):
    """"this function will take the sentence, generate tokens,
    randomly mask 40% tokens for encoder. unmasked 60% tokens are masked in decoder.
    """
    # Tokenize the sentence
    tokenized_sentence = self.tokenizer(sentence, return_tensors="pt",truncation=True,max_length = 512, pad_to_max_length=True, add_special_tokens=True).input_ids[0]
    label = tokenized_sentence.clone()

    # Calculate the number of tokens to mask
    num_tokens_to_mask = int(mask_percentage * len(tokenized_sentence))

    # Randomly choose indices to mask
    encoder_masked_indices = random.sample(range(1, len(tokenized_sentence) - 1), num_tokens_to_mask)
    decoder_masked_indices = list(set(np.arange(0, len(tokenized_sentence) - 1)) - set(encoder_masked_indices))

    # Mask the chosen tokens in encoder
    for index in encoder_masked_indices:
        tokenized_sentence[index] = self.tokenizer.convert_tokens_to_ids(f"<masked_token>")


    #mask rest of the tokens in decoder
    for index in decoder_masked_indices:
        label[index] = self.tokenizer.convert_tokens_to_ids(f"<masked_token>")

    encoder_text = self.tokenizer.decode(tokenized_sentence.flatten())
    decoder_text = self.tokenizer.decode(label.flatten())

    return encoder_text, decoder_text

  def mask_split_words(self, sentence, mask_percentage=0.4):
    """"this function will take the sentence, split it into word tokens,
    randomly mask 40% tokens for encoder. unmasked 60% tokens are masked in decoder.
    """
    # Tokenize the sentence
    tokenized_sentence = sentence.split(' ')
    label = tokenized_sentence.copy()

    # Calculate the number of tokens to mask
    num_tokens_to_mask = int(mask_percentage * len(tokenized_sentence))


    # Randomly choose indices to mask
    encoder_masked_indices = random.choices(range(0, len(tokenized_sentence)), k=num_tokens_to_mask)
    try:
      decoder_masked_indices = list(set(np.arange(0, len(tokenized_sentence))) - set(encoder_masked_indices))
    except:
      decoder_masked_indices = list(set(encoder_masked_indices) - set(np.arange(0, len(tokenized_sentence))))

    # Mask the chosen tokens in encoder
    for index in encoder_masked_indices:
        tokenized_sentence[index] = "<masked_token>"

    #mask rest of the tokens in decoder
    for index in decoder_masked_indices:
        label[index] = "<masked_token>"

    return tokenized_sentence, label

  def generate_data(self, splitting_method, masked_perc, masking_method):
    """This funtion will process the raw text, clean it, split it into long or short sentences as specified,
    mask these sentences, and generate unsupervised dataframe with masked encoder and decoder data
    """
    #remove new line characters
    text = self.corpus.replace('\n',' ')

    #remove special characters
    pattern = re.compile('[^a-zA-Z;?.,%\']')
    clean_text = pattern.sub(' ', text)
    clean_text = clean_text.replace('  ',' ')

    #create topic sequences
    text_strings = clean_text.split(splitting_method)

    #create masks for each topic strings
    encoder_data = []
    decoder_data = []
    for string in text_strings:
      if masking_method == 0:
        encoder, decoder = self.mask_tokens(string, mask_percentage = masked_perc)
      else :
        encoder, decoder = self.mask_split_words(string, mask_percentage = masked_perc)
        encoder = (' ').join(encoder)
        decoder = (' ').join(decoder)
      encoder_data.append(encoder)
      decoder_data.append(decoder)

    #create final dataframe
    df = pd.DataFrame({'Encoder': encoder_data, 'Decoder': decoder_data})
    return df

  def create_dataset(self, masking_method):
    """ create a final dataset which contains both short and long sentences with varying degree of masking in encoder"""
    df1 = self.generate_data( '%', 0.1, masking_method)
    df2 = self.generate_data('%', 0.3, masking_method)
    df3 = self.generate_data('%', 0.5, masking_method)
    df4 = self.generate_data('.',0.3, masking_method)
    unsupervised_dataset = pd.concat([df1,df2,df3,df4],axis=0)
    return unsupervised_dataset


In [8]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

(…)ace.co/t5-base/resolve/main/spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…)e.co/t5-base/resolve/main/tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

(…)face.co/t5-base/resolve/main/config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [21]:
DataGenerator = MaskData(corpus, tokenizer)

## create dataset with masked tokens
masked_tokens_data = DataGenerator.create_dataset(0)

## create dataset with masked words
masked_words_data = DataGenerator.create_dataset(1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

In [24]:
masked_tokens_data.to_csv('unsupervised_dataset_masked_tokens.csv')
masked_words_data.to_csv('unsupervised_dataset_masked_words.csv')